## 훈련 데이터에 포함된 노이즈까지 학습한 상태, 과적합을 막을 수 있는 여러가지 방법을 알아보자.

### 1. 데이터의 양을 늘리기

데이터의 양을 늘릴수록 모델은 데이터의 일반적인 패턴을 학습하여 과적합을 방지할 수 있다. 데이터의 양이 적을때면 의도적으로 데이터의 양을 늘리기도 하는데, 이를 Data Augmentation이라고 한다.

### 2. 모델의 복잡도 줄이기

인공 신경망의 복잡도는 은닉층(hidden layer)의 수나 매개변수의 수 등으로 결정된다. 과적합 방지를 위해 인공 신경망의 복잡도를 줄이는 것 또한 대책이 될 수 있다.

In [ ]:
from torch import nn

class Architecture1(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Architecture1, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

위의 신경망이 과적합 현상을 보인다면, 다음과 같이 복잡도를 줄여볼 수 있다.

In [ ]:
class Architecture1(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Architecture1, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

인경 신경망에서는 모델에 있는 매개변수들의 수를 모델의 수용력(capacity)라고 하기도 한다.

### 3. 가중치 규제(Regularization) 적용하기

복잡한 모델이 간단한 모델보다 과적합될 가능성이 높다.  
그리고 간단한 모델은 적은 수의 매개변수를 가진 모델을 말한다.  
복잡한 모델을 좀 더 간단하게 하는 방법으로 가중치 규제(Regularization)이 있다.

L1 규제 : 모든 가중치 w에 대해서 절댓값 합계를 비용함수에 추가한다. L1 norm  
L2 규제 : 모든 가중치 w에 대해서 제곱합을 비용함수에 추가한다. L2 norm

여기서 람다는 규제의 강도를 정하는 하이퍼파라미터이다.  
람다가 크다면 모델이 훈련 데이터에 대해서 적절한 파라미터를 찾는 것보다 규제를 위해 추가된 항들을 작게 유지하는 것을 우선한다는 의미가 된다.  

이 두 식 모두 비용함수를 최소화하기 위해서는 가중치 w들의 값이 작아져야 한다는 특징이 있다.  
예를 들어 L1 규제를 보자. L1 규제를 사용하면, 비용함수가 최소가 되게 하는 가중치와 편향을 찾는 동시에 가중치들의 절댓값의 합 또한 최소가 되어야한다. 이렇게 되면, 가중치 w의 값들은 0 또는 0에 가까이 작아져야 하므로 어떤 특성들은 모델을 만들 때 거의 사용되지 않게 된다.  
L2 규제는 가중치의 제곱을 최소화하므로, w의 값이 완전히 0이 되기 보다는 0에 가까워지는 경향을 띈다.  

L1 규제는 어떤 특성들이 모델에 영향을 주고 있는지를 정확히 판단하고자 할때 유용하다.  
경험적으로는 L2 규제가 더 잘 동작하므로 L2 규제를 더 권장한다.  
인공 신경망에서 L2 규제는 가중치 감쇠(weight decay)라고도 부른다.  

파이토치에서는 옵티마지어의 weight_decay 매개변수를 설정함으로써 L2 규제를 적용한다. weight_decay 매개변수의 기본값은 0이다.  
weight_decay 매개변수에 다른 값을 설정할 수도 있다.

In [ ]:
model = Architecture1(10,20,2)
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4,weight_decay = 1e-5)

### 4. 드롭아웃(Dropout)

학습 과정에서 신경망 일부를 사용하지 않는 방법이다.
비율을 정하면 랜덤으로 그 비율만큼의 뉴런을 학습에 사용하지 않는 것이다. 학습 시에 인공 신경망이 특정 뉴런 또는 특정 조합에 너무 의존적이게 되는 것을 방지하고, 매번 랜덤 선택으로 뉴런들을 선택하지 않으므로 서로 다른 신경망들을 앙상블하여 사용하는 것과 같은 효과를 내어 과적합을 방지한다.